Here are the steps for building your first random forest model using Scikit-Learn:

    Set up your environment.
    Import libraries and modules.
    Load red wine data.
    Split data into training and test sets.
    Declare data preprocessing steps.
    Declare hyperparameters to tune.
    Tune model using cross-validation pipeline.
    Refit on the entire training set.
    Evaluate model pipeline on test data.
    Save model for further use.


    To begin, let's import numpy, which provides support for more efficient numerical computation:

import numpy as np

    Next, we'll import Pandas, a convenient library that supports dataframes . Pandas is technically optional because Scikit-Learn can handle numerical matrices directly, but it'll make our lives easier:

import pandas as pd

    Now it's time to start importing functions for machine learning. The first one will be the train_test_split() function from the model_selection module. As its name implies, this module contains many utilities that will help us choose between models.
	
from sklearn.model_selection import train_test_split

    Next, we'll import the entire preprocessing module. This contains utilities for scaling, transforming, and wrangling data.

from sklearn import preprocessing

    We can import the random forest family like so:
	
from sklearn.ensemble import RandomForestRegressor

    For the scope of this tutorial, we'll only focus on training a random forest and tuning its parameters. We'll have another detailed tutorial for how to choose between model families.

    For now, let's move on to importing the tools to help us perform cross-validation.
	
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

    Next, let's import some metrics we can use to evaluate our model performance later.
	
from sklearn.metrics import mean_squared_error, r2_score

    And finally, we'll import Import module for saving scikit-learn models
	
from sklearn.externals import joblib

    Joblib is an alternative to Python's pickle package, and we'll use it because it's more efficient for storing large numpy arrays.

    Now Load wine data from remote URL

dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'

data = pd.read_csv(dataset_url)

Now let's take a look at the first 5 rows of data

In [5]:
# Import libraries and modules
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
 
# Load red wine data.
dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(dataset_url)

In [6]:
data.head()

,"fixed acidity;""volatile acidity"";""citric acid"";""residual sugar"";""chlorides"";""free sulfur dioxide"";""total sulfur dioxide"";""density"";""pH"";""sulphates"";""alcohol"";""quality"""
0,7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5
1,7.8;0.88;0;2.6;0.098;25;67;0.9968;3.2;0.68;9.8;5
2,7.8;0.76;0.04;2.3;0.092;15;54;0.997;3.26;0.65;...
3,11.2;0.28;0.56;1.9;0.075;17;60;0.998;3.16;0.58...
4,7.4;0.7;0;1.9;0.076;11;34;0.9978;3.51;0.56;9.4;5


that looks really messy. Upon further inspection, it looks like the CSV file is actually using semicolons to separate the data. That's annoying, but easy to fix:

In [7]:
data = pd.read_csv(dataset_url, sep=';')

In [8]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Great, that's much nicer. Now, let's take a look at the data.

In [10]:
data.shape

(1599, 12)

We have 1,599 samples and 12 features, including our target feature. We can easily print some summary statistics.

In [11]:
print(data.describe())

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    1599.000000       1599.000000  1599.000000     1599.000000   
mean        8.319637          0.527821     0.270976        2.538806   
std         1.741096          0.179060     0.194801        1.409928   
min         4.600000          0.120000     0.000000        0.900000   
25%         7.100000          0.390000     0.090000        1.900000   
50%         7.900000          0.520000     0.260000        2.200000   
75%         9.200000          0.640000     0.420000        2.600000   
max        15.900000          1.580000     1.000000       15.500000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  1599.000000          1599.000000           1599.000000  1599.000000   
mean      0.087467            15.874922             46.467792     0.996747   
std       0.047065            10.460157             32.895324     0.001887   
min       0.012000             1.000000         

Here's the list of all the features:

    quality (target)
    fixed acidity
    volatile acidity
    citric acid
    residual sugar
    chlorides
    free sulfur dioxide
    total sulfur dioxide
    density
    pH
    sulphates
    alcohol

All of the features are numeric, which is convenient. However, they have some very different scales, so let's make a mental note to standardize the data later.

Splitting the data into training and test sets at the beginning of your modeling workflow is crucial for getting a realistic estimate of your model's performance.

First, let's separate our target (y) features from our input (X) features:

In [12]:
y = data.quality
X = data.drop('quality', axis=1)

This allows us to take advantage of Scikit-Learn's useful train_test_split function:

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=y)

As you can see, we'll set aside 20% of the data as a test set for evaluating our model. We also set an arbitrary "random state" (a.k.a. seed) so that we can reproduce our results.

Finally, it's good practice to stratify your sample by the target variable. This will ensure your training set looks similar to your test set, making your evaluation metrics more reliable.

we made the mental note to standardize our features because they were on different scales.

Standardization is a common requirement for machine learning tasks. Many algorithms assume that all features are centered around zero and have approximately the same variance.

Scikit-Learn makes data preprocessing a breeze. For example, it's pretty easy to simply scale a dataset:

    Lazy way of scaling data

	
X_train_scaled = preprocessing.scale(X_train)
print(X_trained_scaled)

    You can confirm that the scaled dataset is indeed centered at zero, with unit variance:
	
print(X_train_scaled.mean(axis=0))
print(X_train_scaled.std(axis=0))

    Great, but why did we say that we won't use this code?

    The reason is that we won't be able to perform the exact same transformation on the test set.

    Sure, we can still scale the test set separately, but we won't be using the same means and standard deviations as we used to transform the training set.

    In other words, that means it wouldn't be a fair representation of how the model pipeline, include the preprocessing steps, would perform on brand new data.

    Now, here's the preprocessing code we will use...

    So instead of directly invoking the scale function, we'll be using a feature in Scikit-Learn called the Transformer API. The Transformer API allows you to "fit" a preprocessing step using the training data the same way you'd fit a model...

    ...and then use the same transformation on future data sets!

    Here's what that process looks like:

    Fit the transformer on the training set (saving the means and standard deviations)
    Apply the transformer to the training set (scaling the training data)
    Apply the transformer to the test set (using the same means and standard deviations)

    This makes your final estimate of model performance more realistic, and it allows to insert your preprocessing steps into a cross-validation pipeline (more on this in Step 7).

    Here's how you do it:
    Fitting the Transformer API

In [15]:
scaler = preprocessing.StandardScaler().fit(X_train)

Now, the scaler object has the saved means and standard deviations for each feature in the training set.

Let's confirm that worked:

In [16]:
X_train_scaled = scaler.transform(X_train)
 
print(X_train_scaled.mean(axis=0))
print(X_train_scaled.std(axis=0))

[ 1.16664562e-16 -3.05550043e-17 -8.47206937e-17 -2.22218213e-17
  2.22218213e-17 -6.38877362e-17 -4.16659149e-18 -2.54439854e-15
 -8.70817622e-16 -4.08325966e-16 -1.17220107e-15]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


Note how we're taking the scaler object and using it to transform the training set. Later, we can transform the test set using the exact same means and standard deviations used to transform the training set:

In [17]:
X_test_scaled = scaler.transform(X_test)
 
print(X_test_scaled.mean(axis=0))
 
print(X_test_scaled.std(axis=0))

[ 0.02776704  0.02592492 -0.03078587 -0.03137977 -0.00471876 -0.04413827
 -0.02414174 -0.00293273 -0.00467444 -0.10894663  0.01043391]
[1.02160495 1.00135689 0.97456598 0.91099054 0.86716698 0.94193125
 1.03673213 1.03145119 0.95734849 0.83829505 1.0286218 ]


    Notice how the scaled features in the test set are not perfectly centered at zero with unit variance! This is exactly what we'd expect, as we're transforming the test set using the means from the training set, not from the test set itself.

    In practice, when we set up the cross-validation pipeline, we won't even need to manually fit the Transformer API. Instead, we'll simply declare the class object, like so:
	
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))

Now it's time to consider the hyperparameters that we'll want to tune for our model.

What are hyperparameters?

There are two types of parameters we need to worry about: model parameters and hyperparameters. Models parameters can be learned directly from the data (i.e. regression coefficients), while hyperparameters cannot.

Hyperparameters express "higher-level" structural information about the model, and they are typically set before training the model.

Example: random forest hyperparameters.

As an example, let's take our random forest for regression:

Within each decision tree, the computer can empirically decide where to create branches based on either mean-squared-error (MSE) or mean-absolute-error (MAE). Therefore, the actual branch locations are model parameters.

However, the algorithm does not know which of the two criteria, MSE or MAE, that it should use. The algorithm also cannot decide how many trees to include in the forest. These are examples of hyperparameters that the user must set.

We can list the tunable hyperparameters like so:
List tunable hyperparameters
	
print pipeline.get_params()

    # 'randomforestregressor__criterion': 'mse',
    # 'randomforestregressor__max_depth': None,
    # 'randomforestregressor__max_features': 'auto',
    # 'randomforestregressor__max_leaf_nodes': None,

You can also find a list of all the parameters on the RandomForestRegressor documentation page. Just note that when it's tuned through a pipeline, you'll need to prepend  randomforestregressor__ before the parameter name, like in the code above.

Now, let's declare the hyperparameters we want to tune through cross-validation.
Declare hyperparameters to tune
	
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}


As you can see, the format should be a Python dictionary (data structure for key-value pairs) where keys are the hyperparameter names and values are lists of settings to try. The options for parameter values can be found on the documentation page.

Step 7: Tune model using a cross-validation pipeline.

Now we're almost ready to dive into fitting our models. But first, we need to spend some time talking about cross-validation.

This is one of the most important skills in all of machine learning because it helps you maximize model performance while reducing the chance of overfitting.

WTF is cross-validation (CV)?

Cross-validation is a process for reliably estimating the performance of a method for building a model by training and evaluating your model multiple times using the same method.

Practically, that "method" is simply a set of hyperparameters in this context.

These are the steps for CV:

    Split your data into k equal parts, or "folds" (typically k=10).
    Train your model on k-1 folds (e.g. the first 9 folds).
    Evaluate it on the remaining "hold-out" fold (e.g. the 10th fold).
    Perform steps (2) and (3) k times, each time holding out a different fold.
    Aggregate the performance across all k folds. This is your performance metric.

K-Fold Cross-Validation

K-Fold Cross-Validation diagram, courtesy of Wikipedia

Why is cross-validation important in machine learning?

Let's say you want to train a random forest regressor. One of the hyperparameters you must tune is the maximum depth allowed for each decision tree in your forest.

How can you decide?

That's where cross-validation comes in. Using only your training set, you can use CV to evaluate different hyperparameters and estimate their effectiveness.

This allows you to keep your test set "untainted" and save it for a true hold-out evaluation when you're finally ready to select a model.

For example, you can use CV to tune a random forest model, a linear regression model, and a k-nearest neighbors model, using only the training set. Then, you still have the untainted test set to make your final selection between the model families!

So WTF is a cross-validation "pipeline?"

The best practice when performing CV is to include your data preprocessing steps inside the cross-validation loop. This prevents accidentally tainting your training folds with influential data from your test fold.

Here's how the CV pipeline looks after including preprocessing steps:

    Split your data into k equal parts, or "folds" (typically k=10).
    Preprocess k-1 training folds.
    Train your model on the same k-1 folds.
    Preprocess the hold-out fold using the same transformations from step (2).
    Evaluate your model on the same hold-out fold.
    Perform steps (2) - (5) k times, each time holding out a different fold.
    Aggregate the performance across all k folds. This is your performance metric.

Fortunately, Scikit-Learn makes it stupidly simple to set this up:
Sklearn cross-validation with pipeline

	
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
    # Fit and tune model
clf.fit(X_train, y_train)

Yes, it's really that easy. GridSearchCV essentially performs cross-validation across the entire "grid" (all possible permutations) of hyperparameters.

It takes in your model (in this case, we're using a model pipeline), the hyperparameters you want to tune, and the number of folds to create.

Obviously, there's a lot going on under the hood. We've included the pseudo-code above, and we'll cover writing cross-validation from scratch in a separate guide.

Now, you can see the best set of parameters found using CV:

	
print clf.best_params_

    # {'randomforestregressor__max_depth': None, 'randomforestregressor__max_features': 'auto'}

Interestingly, it looks like the default parameters win out for this data set.

*Tip: It turns out that in practice, random forests don't actually require a lot of tuning. They tend to work pretty well out-of-the-box with a reasonable number of trees. Even so, these same steps can be used when building any type of supervised learning model.
Step 8: Refit on the entire training set.

After you've tuned your hyperparameters appropriately using cross-validation, you can generally get a small performance improvement by refitting the model on the entire training set.

Conveniently, GridSearchCV from sklearn will automatically refit the model with the best set of hyperparameters using the entire training set.

This functionality is ON by default, but you can confirm it:
Confirm model will be retrained

	
print clf.refit
    # True

Now, you can simply use the  clf object as your model when applying it to other sets of data. That's what we'll be doing in the next step.
Step 9: Evaluate model pipeline on test data.

Alright, we're in the home stretch!

This step is really straightforward once you understand that the  clf object you used to tune the hyperparameters can also be used directly like a model object.

Here's how to predict a new set of data:
Predict a new set of data

	
y_pred = clf.predict(X_test)

Now we can use the metrics we imported earlier to evaluate our model performance.

	
print r2_score(y_test, y_pred)
    # 0.45044082571584243
 
print mean_squared_error(y_test, y_pred)
    # 0.35461593750000003

Great, so now the question is... is this performance good enough?

Well, the rule of thumb is that your very first model probably won't be the best possible model. However, we recommend a combination of three strategies to decide if you're satisfied with your model performance.

    Start with the goal of the model. If the model is tied to a business problem, have you successfully solved the problem?
    Look in academic literature to get a sense of the current performance benchmarks for specific types of data.
    Try to find low-hanging fruit in terms of ways to improve your model.

There are various ways to improve a model. We'll have more guides that go into detail about how to improve model performance, but here are a few quick things to try:

    Try other regression model families (e.g. regularized regression, boosted trees, etc.).
    Collect more data if it's cheap to do so.
    Engineer smarter features after spending more time on exploratory analysis.
    Speak to a domain expert to get more context (...this is a good excuse to go wine tasting!).

As a final note, when you try other families of models, we recommend using the same training and test set as you used to fit the random forest model. That's the best way to get a true apples-to-apples comparison between your models.
Step 10: Save model for future use.

Great job completing this tutorial!

You've done the hard part, and deserve another glass of wine. Maybe this time you can use your shiny new predictive model to select the bottle.

But before you go, let's save your hard work so you can use the model in the future. It's really easy to do so:
Save model to a .pkl file

	
joblib.dump(clf, 'rf_regressor.pkl')

And that's it. When you want to load the model again, simply use this function:
Load model from .pkl file
	
clf2 = joblib.load('rf_regressor.pkl')
 
    # Predict data set using loaded model
clf2.predict(X_test)

Congratulations, you've reached the end of this tutorial!

We've just completed a whirlwind tour of Scikit-Learn's core functionality, but we've only really scratched the surface. Hopefully you've gained some guideposts to further explore all that sklearn has to offer.

For continued learning, we recommend studying other examples in sklearn.
The complete code, from start to finish.

Here's all the code in one place, in a single script.

In [2]:
# Import libraries and modules
import numpy as np
import pandas as pd
 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib 
 
# Load red wine data.
dataset_url = 'http://mlr.cs.umass.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(dataset_url, sep=';')
 
# Split data into training and test sets
y = data.quality
X = data.drop('quality', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123, 
                                                    stratify=y)
 
# Declare data preprocessing steps
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))
 
# Declare hyperparameters to tune
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}
 
# Tune model using cross-validation pipeline
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
clf.fit(X_train, y_train)
 
# Refit on the entire training set
# No additional code needed if clf.refit == True (default is True)
 
# Evaluate model pipeline on test data
pred = clf.predict(X_test)
print(r2_score(y_test, pred))
print(mean_squared_error(y_test, pred))
 
# Save model for future use
joblib.dump(clf, 'rf_regressor.pkl')
# To load: clf2 = joblib.load('rf_regressor.pkl')

C:\Users\SP\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


0.4683818633089172
0.34303906249999994


['rf_regressor.pkl']